## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import random
import re
from collections import Counter
import pymorphy2
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD
import numpy as np
import warnings

### Читаем train и test

In [6]:
lines = list(open('./news_train.txt', 'r', encoding='utf-8'))

with open('./news_test.txt', 'w', encoding='utf-8') as f:
    for line in lines[12000:15000]:
        f.write(line)

In [7]:
train = lines[:10000]
test = lines[10000:]

In [8]:
dict_train = {'labels': [], 'headings': [], 'texts': []}
dict_test = {'labels': [], 'headings': [], 'texts': []}

for line in train:
    label, heading, text = line.split('\t')
    dict_train['labels'].append(label)
    dict_train['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_train['texts'].append(re.findall(r'\b\w+\b', text.lower()))

for line in test:
    label, heading, text = line.split('\t')
    dict_test['labels'].append(label)
    dict_test['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_test['texts'].append(re.findall(r'\b\w+\b', text.lower()))

In [9]:
morph = pymorphy2.MorphAnalyzer()

dict_train['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_train['texts']]

In [10]:
dict_test['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_test['texts']]

In [11]:
model = Word2Vec(sentences=dict_train['texts'], workers=4)

### Посмотрим насколько хорошо обучились эмбединги

In [12]:
model.wv.most_similar(positive=['зима'])

[('электропоезд', 0.7933211922645569),
 ('зимой', 0.7791734933853149),
 ('осенний', 0.7771607041358948),
 ('глядеть', 0.7674463391304016),
 ('бесконечность', 0.7592796683311462),
 ('петербуржец', 0.7563175559043884),
 ('фламенго', 0.7550725936889648),
 ('тягач', 0.7546178102493286),
 ('шаттл', 0.7468329668045044),
 ('улучшиться', 0.7460943460464478)]

In [13]:
model.wv.most_similar(positive=['спорт'])

[('р', 0.8193103671073914),
 ('экспресс', 0.7762881517410278),
 ('мутко', 0.7509728074073792),
 ('виталий', 0.7504749894142151),
 ('футбол', 0.6780646443367004),
 ('тасс', 0.670963704586029),
 ('итар', 0.6644529104232788),
 ('федерация', 0.6332300901412964),
 ('атор', 0.6295084357261658),
 ('александр', 0.622334361076355)]

### Алгоритм классификации

In [14]:
X_train = [' '.join(text) for text in dict_train['texts']]
y_train = dict_train['labels']
X_test = [' '.join(text) for text in dict_test['texts']]
y_test = dict_test['labels']

In [15]:
tfidf = TfidfVectorizer()

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [21]:
RandomSearch_logreg = RandomizedSearchCV(LogisticRegression(), param_distributions={'C': np.arange(0.01, 1.01, 0.01)})
RandSearch_naivebayes = RandomizedSearchCV(MultinomialNB(), param_distributions={'alpha': np.arange(0.5, 1.51, 0.01)})

In [17]:
RandomSearch_logreg.fit(X_train_vec, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])})

In [22]:
RandSearch_naivebayes.fit(X_train_vec, y_train)

RandomizedSearchCV(estimator=MultinomialNB(),
                   param_distributions={'alpha': array([0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
       0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71,
       0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82,
       0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93,
       0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04,
       1.05, 1.06, 1.07, 1.08, 1.09, 1.1 , 1.11, 1.12, 1.13, 1.14, 1.15,
       1.16, 1.17, 1.18, 1.19, 1.2 , 1.21, 1.22, 1.23, 1.24, 1.25, 1.26,
       1.27, 1.28, 1.29, 1.3 , 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37,
       1.38, 1.39, 1.4 , 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48,
       1.49, 1.5 ])})

### Посмотрим на получившиеся метрики качества

In [23]:
y_pred_lreg = RandCV_logreg.best_estimator_.predict(X_test_vec)
print('Метрики logreg')
print('Precision: ', precision_score(y_pred_lreg, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_lreg, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_lreg, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_lreg, y_test))

Метрики logreg
Precision:  0.8835935554826585
Recall:  0.863
F1 score:  0.8693407062674021
Accuracy:  0.863


In [25]:
y_pred_nb = RandSearch_naivebayes.predict(X_test_vec)
print('Метрики naive bayes')
print('Precision: ', precision_score(y_pred_nb, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_nb, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_nb, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_nb, y_test))

Метрики naive bayes
Precision:  0.870979534170263
Recall:  0.7944
F1 score:  0.8234675392394345
Accuracy:  0.7944


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Честно говоря, очень не хочется с помощью своей машины обучать BERT. Вот пример того, как я реализовывал классификацию на BERT для датасетов с каггла

https://colab.research.google.com/drive/1nRFMVkzVrnlekeRHwOP_x-tZ8ev19gj3?usp=sharing